## First we need to create our query and passage and store them in .json

In [1]:
import jsonlines
query = 'Distilled water'
passage1 = 'Distilled water is water that has been boiled into vapor and condensed back into liquid in a separate container. Impurities in the original water that do not boil below or near the boiling point of water remain in the original container. Thus, distilled water is a type of purified water.'
passage2 = "An electric discharge is the release and transmission of electricity in an applied electric field through a medium such as a gas."

In [2]:
!mkdir test
with jsonlines.open('test/test_query.json', 'w') as writer:
    writer.write({'text_id':0, 'text':query})
with jsonlines.open('test/test_passage.json', 'w') as writer:
    writer.write({'text_id':0, 'text':passage1})
    writer.write({'text_id':1, 'text':passage2})

A subdirectory or file test already exists.


## Encode passage

In [10]:
!python -m tevatron.driver.encode \
  --output_dir ./retriever_model \
  --model_name_or_path Luyu/co-condenser-marco \
  --per_device_eval_batch_size 128 \
  --encode_in_path test/test_passage.json \
  --encoded_save_path test/test_passage.pt

03/14/2022 16:11:04 - INFO - tevatron.modeling -   try loading tied weight
03/14/2022 16:11:04 - INFO - tevatron.modeling -   loading model weight from Luyu/co-condenser-marco
Some weights of the model checkpoint at Luyu/co-condenser-marco were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
S

## Encode query

In [11]:
!python -m tevatron.driver.encode --output_dir ./coCondenser/retriever_model \
    --model_name_or_path Luyu/co-condenser-marco   \
     --q_max_len 32 --encode_is_qry \
    --per_device_eval_batch_size 128 \
    --encode_in_path test/test_query.json \
    --encoded_save_path test/test_query.pt

03/11/2022 02:00:36 - INFO - tevatron.modeling -   try loading tied weight
03/11/2022 02:00:36 - INFO - tevatron.modeling -   loading model weight from Luyu/co-condenser-marco
Some weights of the model checkpoint at Luyu/co-condenser-marco were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
S


Dataset json downloaded and prepared to C:\Users\Amanco\.cache\huggingface\datasets\json\default-80e0c2ae213494a4\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 500.57it/s]

100%|██████████| 1/1 [00:00<00:00, 500.27it/s]

100%|██████████| 1/1 [00:00<00:00, 13.71it/s]


## Load the embedding vectors for query and passage and do cosine similarity

In [5]:
# Tevatron's repository recently changed their encodings to numpy instead of torch
import numpy as np
query_embed = np.load('test/test_query.pt', allow_pickle=True)
passage_embed = np.load('test/test_passage.pt', allow_pickle=True)
print(query_embed[0].shape)
print(passage_embed[0].shape)

(1, 768)
(2, 768)


In [6]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(query_embed[0], passage_embed[0])

array([[0.950768  , 0.87579244]], dtype=float32)